In [1]:
with open('the-verdict.txt', 'r') as f:
    book_text = f.read()

In [2]:
import torch
inputs = torch.tensor(
 [[0.43, 0.15, 0.89], # Your (x^1)
 [0.55, 0.87, 0.66], # journey (x^2)
 [0.57, 0.85, 0.64], # starts (x^3)
 [0.22, 0.58, 0.33], # with (x^4)
 [0.77, 0.25, 0.10], # one (x^5)
 [0.05, 0.80, 0.55]] # step (x^6)
)

In [3]:
query = inputs[1]
att_scores = torch.empty(inputs.shape[0])
att_scores

tensor([2.9614e-09, 0.0000e+00, 0.0000e+00, 0.0000e+00, 3.4960e-41, 0.0000e+00])

In [4]:
for i, x_i in enumerate(inputs):
    att_scores[i] = torch.dot(x_i, query)
print(att_scores)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


In [5]:
att_scores_normalized = att_scores / att_scores.sum()
print(att_scores_normalized)
print(att_scores_normalized.sum())

tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
tensor(1.0000)


In [6]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

In [7]:
att_scores_smax_naive = softmax_naive(att_scores)
print(att_scores_smax_naive)
print(att_scores_smax_naive.sum())

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
tensor(1.)


In [8]:
att_scores_smax = torch.softmax(att_scores, dim=0)
print(att_scores_smax)
print(att_scores_smax.sum())

tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
tensor(1.)


In [9]:
ctx_scores = torch.zeros(query.shape)
for i, x_i in enumerate(inputs):
    ctx_scores += x_i * att_scores_smax[i]
print(ctx_scores)

tensor([0.4419, 0.6515, 0.5683])


## Computing context vectors for all inputs

In [10]:
att_scores = torch.empty((inputs.shape[0], inputs.shape[0]))
att_scores.shape

torch.Size([6, 6])

In [11]:
# Way 1 to multiple a matrix by its transpose
for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        att_scores[i, j] = torch.dot(x_i, x_j)
print(att_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [12]:
# Way 2 to do the same
att_scores2 = torch.empty((inputs.shape[0], inputs.shape[0]))
att_scores2 = inputs @ inputs.T
print(att_scores2)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


In [13]:
# softmax along the last dim (-1)
att_weights = torch.softmax(att_scores2, dim=-1)
print(att_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


In [14]:
print(att_weights[0].sum())
print(att_weights.sum(dim=-1))

tensor(1.0000)
tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


In [15]:
all_ctx_weights = att_weights @ inputs
print(all_ctx_weights)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


In [16]:
# verification
ctx_scores - all_ctx_weights[1]

tensor([0., 0., 0.])

# Self attention with trainable weights

In [17]:
d2 = inputs[1]
din = d2.shape[0]
dout = 2

In [18]:
din

3

In [19]:
# q = query, k = key, v = value
torch.manual_seed(123)
Wq = torch.nn.Parameter(torch.rand(din, dout), requires_grad=False)
Wk = torch.nn.Parameter(torch.rand(din, dout), requires_grad=False)
Wv = torch.nn.Parameter(torch.rand(din, dout), requires_grad=False)

In [20]:
dq = d2 @ Wq
dk = d2 @ Wk
dv = d2 @ Wv

In [21]:
print(dq, ' === ', dk, ' === ', dv)

tensor([0.4306, 1.4551])  ===  tensor([0.4433, 1.1419])  ===  tensor([0.3951, 1.0037])


In [22]:
keys = inputs @ Wk
values = inputs @ Wv
print(keys.shape, ' === ', values.shape)

torch.Size([6, 2])  ===  torch.Size([6, 2])


In [23]:
att_scores2 = dq @ keys.T
print(att_scores2.shape, ' === ', (keys @ dq).shape)
print(att_scores2)

torch.Size([6])  ===  torch.Size([6])
tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


In [24]:
w22 = dq.dot(keys[1]);
print("w22 = ", w22)
print(w22 - att_scores2[1])

w22 =  tensor(1.8524)
tensor(0.)


In [25]:
dimk = keys.shape[-1]
print(dimk)

2


In [26]:
att_weights2 = torch.softmax(att_scores2 / dimk**0.5, dim=-1)
print(att_weights2, ' === ', att_weights2.sum(dim=-1))

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])  ===  tensor(1.)


In [27]:
ctx_vec2  = att_weights2 @ values;
print(ctx_vec2)

tensor([0.3061, 0.8210])


# Self Attention in one class

In [41]:
class SelfAttentionV1(torch.nn.Module):
    def __init__(self, din, dout):
        super().__init__()
        self.Wq = torch.nn.Parameter(torch.rand(din, dout))
        self.Wk = torch.nn.Parameter(torch.rand(din, dout))
        self.Wv = torch.nn.Parameter(torch.rand(din, dout))

    def forward(self, x):
        q = x @ self.Wq;
        k = x @ self.Wk;
        v = x @ self.Wv;
        att_scores = q @ k.T;
        ws = torch.softmax(att_scores / k.shape[-1]**0.5, dim=-1)
        ctx = ws @ v
        return ctx

In [51]:
torch.manual_seed(123)
sa1 = SelfAttentionV1(din, dout)
ctxvec = sa1(inputs)
print(ctxvec)

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


In [52]:
# verification
ctx_vec2 - ctxvec[1]

tensor([0., 0.], grad_fn=<SubBackward0>)

In [53]:
class SelfAttentionV2(torch.nn.Module):
    def __init__(self, din, dout, kv_bias=False):
        super().__init__()
        self.Wq = torch.nn.Linear(din, dout, bias=kv_bias)
        self.Wk = torch.nn.Linear(din, dout, bias=kv_bias)
        self.Wv = torch.nn.Linear(din, dout, bias=kv_bias)

    def forward(self, x):
        q = self.Wq(x);
        k = self.Wk(x);
        v = self.Wv(x);
        att_scores = q @ k.T;
        ws = torch.softmax(att_scores / k.shape[-1]**0.5, dim=-1)
        ctx = ws @ v
        return ctx

In [54]:
torch.manual_seed(789)
sa2 = SelfAttentionV2(din, dout)
ctxvec = sa2(inputs)
print(ctxvec)

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


# Exercise 3.1

In [61]:
sa2.Wq.weight.shape, sa1.Wq.shape

(torch.Size([2, 3]), torch.Size([3, 2]))

In [64]:
sa1.Wq = torch.nn.Parameter(sa2.Wq.weight.T)

In [65]:
sa1.Wq = torch.nn.Parameter(sa2.Wq.weight.T)
sa1.Wk = torch.nn.Parameter(sa2.Wk.weight.T)
sa1.Wv = torch.nn.Parameter(sa2.Wv.weight.T)
ctxvec = sa1(inputs)
print(ctxvec)

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


# Causal attention

In [66]:
queries = sa2.Wq(inputs)
keys = sa2.Wk(inputs)
att_scrs = queries @ keys.T
weights = torch.softmax(att_scrs / keys.shape[1]**0.5, dim=-1)
print(weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [69]:
ctx_len = weights.shape[0]
print(ctx_len)

6


In [71]:
diagmask = torch.tril(torch.ones(ctx_len, ctx_len))
print(diagmask)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


In [72]:
# Tril the weights directly
print(torch.tril(weights))

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<TrilBackward0>)


In [73]:
masked_w = diagmask * weights
print(masked_w)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


In [75]:
rsum = masked_w.sum(dim=-1, keepdim=True)
masked_normed_w = masked_w / rsum
print(masked_normed_w)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


In [77]:
rsum1 = masked_w.sum(dim=-1, keepdim=True)
rsum2 = masked_w.sum(dim=-1, keepdim=False)
print(rsum1.shape, ' === ', rsum2.shape)

torch.Size([6, 1])  ===  torch.Size([6])


In [89]:
# My way to get -inf masking
mask2 = torch.tril(torch.ones(ctx_len, ctx_len)) + torch.triu(torch.ones(ctx_len, ctx_len)*(-torch.inf), diagonal=1)
print(mask2)

tensor([[1., -inf, -inf, -inf, -inf, -inf],
        [1., 1., -inf, -inf, -inf, -inf],
        [1., 1., 1., -inf, -inf, -inf],
        [1., 1., 1., 1., -inf, -inf],
        [1., 1., 1., 1., 1., -inf],
        [1., 1., 1., 1., 1., 1.]])


In [106]:
masked_w2 = mask2 * torch.abs(att_scrs)
print(masked_w2)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MulBackward0>)


In [107]:
masked_normed_w2 = torch.softmax(masked_w2 / keys.shape[-1]**0.5, dim=1)
print(masked_normed_w2)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


In [101]:
# books way
mask3 = torch.triu(torch.ones(ctx_len, ctx_len), diagonal=1)
masked_w3 = att_scrs.masked_fill(mask3.bool(), -torch.inf)
print(masked_w3)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


In [102]:
masked_normed_w3 = torch.softmax(masked_w3 / keys.shape[-1]**0.5, dim=1)
print(masked_normed_w3)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


## Dropout

In [109]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5)
print(dropout)
ones = torch.ones(6, 6)
print(ones)
print(dropout(ones))

Dropout(p=0.5, inplace=False)
tensor([[1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.],
        [1., 1., 1., 1., 1., 1.]])
tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [114]:
torch.manual_seed(123)
print(dropout(masked_normed_w3))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7599, 0.6194, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4921, 0.4925, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3966, 0.0000, 0.3775, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.3331, 0.3084, 0.3331, 0.0000]],
       grad_fn=<MulBackward0>)


In [117]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape)

torch.Size([2, 6, 3])


## CausalAttention

In [118]:
class CausalAttention(torch.nn.Module):
    def __init__(self, din, dout, ctxlen, dropout, kv_bias=False):
        super().__init__()
        self.dout = dout
        self.Wq = torch.nn.Linear(din, dout, bias=kv_bias)
        self.Wk = torch.nn.Linear(din, dout, bias=kv_bias)
        self.Wv = torch.nn.Linear(din, dout, bias=kv_bias)
        self.dropout = torch.nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(ctxlen, ctxlen), diagonal=1))

    def forward(self, x):
        b, ntok, din = x.shape
        q = self.Wq(x);
        k = self.Wk(x);
        v = self.Wv(x);
        att_scores = q @ k.transpose(1, 2); # keep batch (dim0) in place
        att_scores.masked_fill_(self.mask.bool()[:ntok, :ntok], -torch.inf);
        ws = torch.softmax(att_scores / k.shape[-1]**0.5, dim=-1)
        ws = self.dropout(ws)
        ctx = ws @ v
        return ctx

In [124]:
# Using this
torch.manual_seed(123)
ctxlen = batch.shape[1]
ca = CausalAttention(din, dout, ctxlen, 0.0)
ca.mask.bool()[:ctxlen, :ctxlen]

tensor([[False,  True,  True,  True,  True,  True],
        [False, False,  True,  True,  True,  True],
        [False, False, False,  True,  True,  True],
        [False, False, False, False,  True,  True],
        [False, False, False, False, False,  True],
        [False, False, False, False, False, False]])

In [125]:
ctx_vecs = ca(batch)
print(ctx_vecs.shape)

torch.Size([2, 6, 2])


# Multi-head attention

In [126]:
class MultiheadAttention(torch.nn.Module):
    def __init__(self, din, dout, ctxlen, dropout, num_heads, kv_bias=False):
        super().__init__()
        self.heads = torch.nn.ModuleList([
            CausalAttention(din, dout, ctxlen, dropout, kv_bias)
            for _ in range(num_heads)
        ])

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)

In [144]:
torch.manual_seed(123)
mha = MultiheadAttention(din, dout, ctxlen, 0.0, num_heads=2)
mh_ctx = mha(batch)
print(mh_ctx.shape)
print(mh_ctx)

torch.Size([2, 6, 4])
tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)


## Exercise 3.2

In [141]:
mha2 = MultiheadAttention(din, 1, ctxlen, 0.5, num_heads=2)
mh_ctx2 = mha2(batch)
print(mh_ctx2.shape)

torch.Size([2, 6, 2])


## Parallel Multihead Attention

In [147]:
class MultiheadAttentionV2(torch.nn.Module):
    def __init__(self, din, dout, ctxlen, dropout, num_heads, kv_bias=False):
        super().__init__()
        assert(dout % num_heads == 0), "d_out MUST be multiple of num_heads"
        self.dout = dout
        self.num_heads = num_heads
        self.head_dim = dout // num_heads
        self.Wq = torch.nn.Linear(din, dout, bias=kv_bias)
        self.Wk = torch.nn.Linear(din, dout, bias=kv_bias)
        self.Wv = torch.nn.Linear(din, dout, bias=kv_bias)
        self.out_proj = torch.nn.Linear(dout, dout)
        self.dropout = torch.nn.Dropout(dropout)
        self.register_buffer('mask', torch.triu(torch.ones(ctxlen, ctxlen), diagonal=1))

    def forward(self, x):
        b, ntok, din = x.shape
        q = self.Wq(x);
        k = self.Wk(x);
        v = self.Wv(x);
        q = q.view(b, ntok, self.num_heads, self.head_dim)
        k = k.view(b, ntok, self.num_heads, self.head_dim)
        v = v.view(b, ntok, self.num_heads, self.head_dim)

        q = q.transpose(1, 2) # make num_heads the dim1 instead of dim2
        k = k.transpose(1, 2)
        v = v.transpose(1, 2)
        
        att_scores = q @ k.transpose(2, 3); # keep batch (dim0) & num_heads (dim1) in place
        att_scores.masked_fill_(self.mask.bool()[:ntok, :ntok], -torch.inf);
        ws = torch.softmax(att_scores / k.shape[-1]**0.5, dim=-1)
        ws = self.dropout(ws)
        ctx = ws @ v
        ctx = ctx.transpose(1, 2) # make num_heads the dim2
        ctx = ctx.contiguous().view(b, ntok, self.dout)  # join dim2 and dim3 into dim2*dim3

        ctx = self.out_proj(ctx) # join everything to dout*dout
        return ctx

In [148]:
# Using this
torch.manual_seed(123)
b, ctxlen, din = batch.shape
dout=2
mhav2 = MultiheadAttentionV2(din, dout, ctxlen, 0.0, 2)
mhav2_ctx = mhav2(batch)
print(mhav2_ctx.shape)

torch.Size([2, 6, 2])


In [149]:
print(mhav2_ctx)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)


# Exercise 3.3

In [151]:
mha_gpt2 = MultiheadAttentionV2(768, 768, 1024, 0.0, num_heads = 12)